# Dataset Pipeline — Two Original Classification Problems (Data Prep Only)

**Scope:** This notebook completes the *dataset portion only* of the project for two original classification problems (no Kaggle/UCI/HF).
It documents **sources**, **ethics**, **quirks**, **descriptive stats & visuals**, **cleaning**, **encoding**, **scaling**, and creates **engineered features** (≥3 per dataset).
*Modeling (Logistic Regression from-scratch, sklearn baselines, SVM, Naive Bayes) will be added later.*

---

## How to Run (Reproducible)
1. Open in **Colab** or local **Jupyter** with internet access.
2. Run cells top to bottom.
3. The notebook downloads raw samples (≥10k rows each) via Socrata SODA API, saves **raw** and **processed** CSVs, and renders EDA plots.
4. Random seeds and library versions are logged.
5. If you encounter API throttling, reduce `PAGE_SIZE` or `MAX_ROWS` in the download cells.


In [ ]:
# === Setup: Versions & Seeds ===
# Purpose: Make runs reproducible and transparent about environment.
import sys, platform, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

np.random.seed(42)
random.seed(42)

print({
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "pandas": pd.__version__,
    "numpy": np.__version__,
    "matplotlib": plt.matplotlib.__version__,
    "requests": requests.__version__,
})

---
# Dataset A — NYC Yellow Taxi Trips (Original Source)

### Source & Documentation
- **Origin:** NYC Taxi & Limousine Commission (TLC) via **NYC Open Data**  
  Public documentation: TLC Trip Record Data  
  Example dataset (2023): NYC Open Data — 2023 Yellow Taxi Trip Data (`4b4i-vvec`)
- **Access method:** Socrata SODA API (`/resource/4b4i-vvec.json`) with `$limit`/`$offset` paging
- **Why it qualifies:** Non-Kaggle/UCI/HF; millions of rows; many features; clear label can be defined

### Proposed Classification Task
Predict **generous tipping**: `high_tip = 1` if `tip_percent ≥ 0.20`, else `0`.  
We limit to **credit-card** trips (`payment_type=1`) because cash tips are not captured in the data.


In [ ]:
# === Download NYC Yellow Taxi sample (>=10k rows) ===
# Purpose: Pull a manageable sample via SODA API to build our dataset locally.
# Guidance: Select a minimal set of fields; use paging due to API row caps.
import time

TLC_BASE = "https://data.cityofnewyork.us/resource/4b4i-vvec.json"  # 2023 Yellow Taxi
PAGE_SIZE = 20000   # reduce if throttled
MAX_ROWS = 20000    # must be >= 3,000

fields = [
    "tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance",
    "PULocationID","DOLocationID","fare_amount","extra","mta_tax","tip_amount",
    "tolls_amount","improvement_surcharge","total_amount","congestion_surcharge",
    "payment_type","ratecode_id"
]
select = ",".join(fields)

rows = []
offset = 0
while len(rows) < MAX_ROWS:
    params = {"$select": select, "$limit": PAGE_SIZE, "$offset": offset}
    r = requests.get(TLC_BASE, params=params, timeout=60)
    r.raise_for_status()
    batch = r.json()
    if not batch:
        break
    rows.extend(batch)
    offset += PAGE_SIZE
    # time.sleep(0.2)  # uncomment if API throttles

taxi_raw = pd.DataFrame(rows)
print("Downloaded rows (NYC taxi):", taxi_raw.shape)
taxi_raw.to_csv("taxi_raw_2023.csv", index=False)

## Data Description & Quirks (Taxi)
**Quirks**
- Impossible/implausible values (negative fare, zero distance, zero duration)  
- Missing `tip_amount` for cash payments → restrict to credit-card records  
- Outliers in `trip_distance`, `duration_min`, `total_amount`  
- Duration depends on timestamp parsing and timezone/DST edges

**Feature & Label Plan**
- `duration_min` = (dropoff − pickup) in minutes  
- `speed_mph` = `trip_distance` / (`duration_min`/60)  
- `tip_percent` = `tip_amount` / (`fare_amount` + 1e-6)  
- `hour`, `dow`, `is_weekend` features from pickup datetime  
- **Target:** `high_tip` = 1 if `tip_percent ≥ 0.20` else 0 (credit-card trips only)


In [ ]:
# === Clean & Engineer (NYC Taxi) ===
# Steps: parse datetimes, cast numerics, filter plausibility, engineer features, build label.
taxi = taxi_raw.copy()

# Parse datetimes
for c in ["tpep_pickup_datetime", "tpep_dropoff_datetime"]:
    taxi[c] = pd.to_datetime(taxi[c], errors="coerce")

# Cast numeric columns
num_cols = ["passenger_count","trip_distance","fare_amount","extra","mta_tax","tip_amount",
            "tolls_amount","improvement_surcharge","total_amount","congestion_surcharge",
            "ratecode_id","payment_type","PULocationID","DOLocationID"]
for c in num_cols:
    taxi[c] = pd.to_numeric(taxi[c], errors="coerce")

# Duration in minutes
taxi["duration_min"] = (taxi["tpep_dropoff_datetime"] - taxi["tpep_pickup_datetime"]).dt.total_seconds() / 60.0

# Plausibility filters
taxi = taxi[
    (taxi["trip_distance"] > 0.1) & (taxi["trip_distance"] < 50) &
    (taxi["duration_min"] >= 1) & (taxi["duration_min"] <= 180) &
    (taxi["fare_amount"] > 2)
].copy()

# Credit-card only
taxi = taxi[(taxi["payment_type"] == 1)].copy()

# Engineered features
taxi["speed_mph"] = taxi["trip_distance"] / (taxi["duration_min"]/60.0)
taxi["tip_percent"] = taxi["tip_amount"] / (taxi["fare_amount"] + 1e-6)
taxi["hour"] = taxi["tpep_pickup_datetime"].dt.hour
taxi["dow"] = taxi["tpep_pickup_datetime"].dt.dayofweek
taxi["is_weekend"] = taxi["dow"].isin([5,6]).astype(int)

# Binary label
taxi["high_tip"] = (taxi["tip_percent"] >= 0.20).astype(int)

# Remove pathological speeds
taxi = taxi[(taxi["speed_mph"] > 1) & (taxi["speed_mph"] < 80)]

print("After cleaning & FE (NYC taxi):", taxi.shape)
taxi.head()

## Descriptive Stats & Visuals (Taxi)
- Summary stats (`describe()`)
- Missingness counts
- Histograms: `trip_distance`, `duration_min`, `tip_percent`
- High-tip rate by hour & weekend


In [ ]:
# === EDA (NYC Taxi) ===
key_cols = ["trip_distance","duration_min","fare_amount","tip_amount","tip_percent","speed_mph"]
print(taxi[key_cols].describe().T)

print("\nMissing values (top 10):\n", taxi.isna().sum().sort_values(ascending=False).head(10))

fig, axes = plt.subplots(1,3, figsize=(12,3))
axes[0].hist(taxi["trip_distance"].dropna(), bins=40)
axes[0].set_title("Trip Distance")
axes[1].hist(taxi["duration_min"].dropna(), bins=40)
axes[1].set_title("Duration (min)")
axes[2].hist(taxi["tip_percent"].dropna().clip(0,1), bins=40)
axes[2].set_title("Tip % (clipped 0-1)")
plt.tight_layout(); plt.show()

hour_rate = taxi.groupby("hour")["high_tip"].mean()
plt.figure(figsize=(5,3)); plt.plot(hour_rate.index, hour_rate.values, marker="o")
plt.title("High Tip Rate by Hour"); plt.xlabel("Hour"); plt.ylabel("Rate"); plt.show()

wk_rate = taxi.groupby("is_weekend")["high_tip"].mean()
plt.figure(figsize=(4,3)); plt.bar(["Weekday","Weekend"], wk_rate.values)
plt.title("High Tip Rate: Weekend vs Weekday"); plt.ylabel("Rate"); plt.show()

## Encoding & Scaling (Taxi)
- High-cardinality IDs (`PULocationID`, `DOLocationID`, `ratecode_id`) are **frequency-encoded** to avoid huge one-hot matrices.  
- Numerical features are standardized for later models.

Output saved as `taxi_clean_features.csv`.


In [ ]:
# === Encode & Scale (NYC Taxi) ===
from sklearn.preprocessing import StandardScaler

taxi_proc = taxi.copy()

# Frequency encoding for high-cardinality columns
for col in ["PULocationID","DOLocationID","ratecode_id"]:
    freq = taxi_proc[col].value_counts(normalize=True)
    taxi_proc[col+"_freq"] = taxi_proc[col].map(freq).fillna(0)

X_cols_taxi = [
    "passenger_count","trip_distance","fare_amount","tolls_amount","total_amount",
    "congestion_surcharge","duration_min","speed_mph","tip_percent","hour","is_weekend",
    "PULocationID_freq","DOLocationID_freq","ratecode_id_freq"
]
y_col_taxi = "high_tip"

num_cols_scale = ["passenger_count","trip_distance","fare_amount","tolls_amount","total_amount",
                  "congestion_surcharge","duration_min","speed_mph","tip_percent","hour",
                  "PULocationID_freq","DOLocationID_freq","ratecode_id_freq"]
scaler = StandardScaler()
taxi_proc[num_cols_scale] = scaler.fit_transform(taxi_proc[num_cols_scale])

taxi_proc[X_cols_taxi + [y_col_taxi]].to_csv("taxi_clean_features.csv", index=False)
print("Saved taxi_clean_features.csv with shape:", taxi_proc[X_cols_taxi + [y_col_taxi]].shape)

---
# Dataset B — City of Chicago Food Inspections (Original Source)

### Source & Documentation
- **Origin:** City of Chicago — Department of Public Health, **official open data**  
  Program page: Food Establishment Inspection Reports  
  API dataset: `https://data.cityofchicago.org/resource/4ijn-s7e5.json`
- **Why it qualifies:** Non-Kaggle/UCI/HF; >3,000 rows; many features; clear label

### Proposed Classification Task
Predict **inspection outcome**: `pass_binary = 1` if `results == "Pass"`, else `0` (`Fail`, `Pass w/ Conditions`, etc.).


In [ ]:
# === Download Chicago Food Inspections sample (>=10k rows) ===
CHI_BASE = "https://data.cityofchicago.org/resource/4ijn-s7e5.json"
PAGE_SIZE = 20000
MAX_ROWS = 20000

fields = [
    "inspection_id","inspection_date","dba_name","aka_name","license_",
    "facility_type","risk","results","violations","address","zip","ward","community_area",
    "latitude","longitude"
]
select = ",".join(fields)

rows = []
offset = 0
while len(rows) < MAX_ROWS:
    params = {"$select": select, "$limit": PAGE_SIZE, "$offset": offset}
    r = requests.get(CHI_BASE, params=params, timeout=60)
    r.raise_for_status()
    batch = r.json()
    if not batch:
        break
    rows.extend(batch)
    offset += PAGE_SIZE

chi_raw = pd.DataFrame(rows)
print("Downloaded rows (Chicago):", chi_raw.shape)
chi_raw.to_csv("chi_food_raw.csv", index=False)

## Data Description & Quirks (Chicago)
**Quirks**
- `results` has multiple categories → binarize to Pass vs Not-Pass  
- `risk` may be missing/inconsistent (Risk 1/2/3)  
- `violations` is semi-structured free text → regex parse for counts  
- Multiple rows per `license_` across time → compute longitudinal features

**Feature & Label Plan**
- `pass_binary` = 1 if `results == 'Pass'`, else 0  
- `violation_count` = number of violation codes parsed from text  
- `viol_text_len` = length of violations narrative  
- `risk_level` = numeric mapping (1=High, 2=Med, 3=Low)  
- `month`, `year`, and `days_since_prev` per facility


In [ ]:
# === Clean & Engineer (Chicago) ===
# Steps: parse dates, build label, parse violations, map risk, add temporal features (including gaps).
import re
chi = chi_raw.copy()

# Parse dates
chi["inspection_date"] = pd.to_datetime(chi["inspection_date"], errors="coerce")

# Binary label
chi["pass_binary"] = (chi["results"].astype(str).str.strip().str.lower() == "pass").astype(int)

# Violation features
def count_violation_codes(s):
    if pd.isna(s) or not isinstance(s, str):
        return 0
    return len(re.findall(r'\b\d{2}\.?\d*\b', s))

chi["violation_count"] = chi["violations"].apply(count_violation_codes)
chi["viol_text_len"] = chi["violations"].fillna("").str.len()

# Risk mapping
risk_map = {"risk 1 (high)": 1, "risk 2 (medium)": 2, "risk 3 (low)": 3}
chi["risk_level"] = chi["risk"].astype(str).str.lower().map(risk_map)

# Temporal features
chi["year"] = chi["inspection_date"].dt.year
chi["month"] = chi["inspection_date"].dt.month

# Days since previous inspection per facility
chi["license_"] = chi["license_"].astype(str)
chi = chi.sort_values(["license_", "inspection_date"])
chi["prev_date"] = chi.groupby("license_")["inspection_date"].shift(1)
chi["days_since_prev"] = (chi["inspection_date"] - chi["prev_date"]).dt.days

print("After cleaning & FE (Chicago):", chi.shape)
chi.head()

## Descriptive Stats & Visuals (Chicago)
- Class balance (Pass rate)  
- Describe `violation_count`, `viol_text_len`, `risk_level`, `days_since_prev`  
- Pass rate by `risk_level` and by `month`


In [ ]:
# === EDA (Chicago) ===
print("Pass rate (mean of pass_binary):", chi["pass_binary"].mean())

desc_cols = ["violation_count","viol_text_len","risk_level","days_since_prev"]
print(chi[desc_cols].describe(include="all").T)

fig, axes = plt.subplots(1,3, figsize=(12,3))
axes[0].hist(chi["violation_count"].dropna(), bins=30)
axes[0].set_title("Violation Count")
axes[1].hist(chi["viol_text_len"].dropna().clip(0,2000), bins=30)
axes[1].set_title("Violations Text Length")
axes[2].hist(chi["days_since_prev"].dropna().clip(0,365*2), bins=30)
axes[2].set_title("Days Since Previous Inspection")
plt.tight_layout(); plt.show()

risk_rate = chi.groupby("risk_level")["pass_binary"].mean()
plt.figure(figsize=(4,3))
plt.bar([str(int(x)) for x in risk_rate.index.fillna(-1)], risk_rate.values)
plt.title("Pass Rate by Risk Level"); plt.xlabel("Risk Level"); plt.ylabel("Pass Rate"); plt.show()

month_rate = chi.groupby("month")["pass_binary"].mean()
plt.figure(figsize=(5,3))
plt.plot(month_rate.index, month_rate.values, marker="o")
plt.title("Pass Rate by Month"); plt.xlabel("Month"); plt.ylabel("Pass Rate"); plt.show()

## Encoding & Scaling (Chicago)
- Minimal numeric feature set is standardized for later models.  
- Optional one-hot encoding for `facility_type` can be added during modeling to control dimensionality.

Output saved as `chi_food_clean_features.csv`.


In [ ]:
# === Encode & Scale (Chicago) ===
from sklearn.preprocessing import StandardScaler

chi_proc = chi.copy()
X_cols_chi = ["violation_count","viol_text_len","risk_level","month","days_since_prev"]
y_col_chi = "pass_binary"

# Median impute numerics before scaling
for c in X_cols_chi:
    if chi_proc[c].isna().any():
        chi_proc[c] = chi_proc[c].fillna(chi_proc[c].median())

scaler_chi = StandardScaler()
chi_proc[X_cols_chi] = scaler_chi.fit_transform(chi_proc[X_cols_chi])

chi_proc[X_cols_chi + [y_col_chi]].to_csv("chi_food_clean_features.csv", index=False)
print("Saved chi_food_clean_features.csv with shape:", chi_proc[X_cols_chi + [y_col_chi]].shape)

---
## Sources & Ethics

**Dataset A — NYC Yellow Taxi Trips**  
- **Source:** NYC Taxi & Limousine Commission (TLC) via NYC Open Data  
- **Access Method:** Socrata SODA API endpoint for 2023 Yellow Taxi Trip Data  
- **License/Terms:** Public data provided for research and transparency by NYC Open Data.  
- **Ethical Considerations:**  
  - Data are de-identified, but location/time patterns can still be sensitive.  
  - To mitigate re-identification risks, exact timestamps and raw coordinates were not used directly; instead, aggregated temporal features (hour, day-of-week) and frequency-encoded pickup/dropoff zones were created.  
  - Analysis is intended for service improvement and academic purposes, not for stigmatizing neighborhoods or individuals.  

**Dataset B — Chicago Food Inspections**  
- **Source:** City of Chicago, Department of Public Health (Food Establishment Inspection Reports)  
- **Access Method:** Socrata SODA API endpoint for Food Inspections data  
- **License/Terms:** Public records provided for transparency and accountability by the City of Chicago.  
- **Ethical Considerations:**  
  - While the dataset is public, results should not be used to stigmatize specific businesses or communities.  
  - Inspections reflect both establishment practices and inspector discretion/scheduling; any predictive modeling should acknowledge potential biases in data collection.  
  - Features engineered (e.g., violation counts, text length, risk level) are intended to support aggregate insights rather than punitive labeling of individual establishments.  

**General Notes**  
- Both datasets are official open data, not sourced from Kaggle, UCI, or similar repositories.  
- Bias and fairness must be considered: historical inequities in inspection or tipping patterns may be reflected in the models.  
- Privacy risks are mitigated by aggregation, feature transformation, and adherence to the terms of each portal.


## Artifacts Produced
- `taxi_raw_2023.csv` — raw sample from NYC TLC (≈20k rows)  
- `taxi_clean_features.csv` — cleaned & engineered feature matrix + label (Taxi)  
- `chi_food_raw.csv` — raw sample from Chicago Food Inspections (≈20k rows)  
- `chi_food_clean_features.csv` — cleaned & engineered feature matrix + label (Chicago)

> These files are ready inputs for the modeling stage of the assignment.
